#        Model-Based Statistical Learning
####  Project 2: analysis of the Enron email network
#### Raphaëlle ROTY

In [1]:
library(tidyverse)
library(tidygraph)
library(igraph)
library(network)
library(ggraph)
library(sna)
library(dplyr)
library(rlist)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



ERROR: ignored

## Part 1: The data

In [3]:
from google.colab import files
upload=files.upload()

ERROR: ignored

In [ ]:
load("D:/OneDrive/Bureau/MSc DATA SCIENCE/M2/Model Based Statistical Learning/Enron.Rdata")
employeelist # the list of the Enron employees and their email addresses,
message # all emails exchanged between 1999 and 2002

Warning message in readChar(con, 5L, useBytes = TRUE):
“cannot open compressed file 'D:/OneDrive/Bureau/MSc DATA SCIENCE/M2/Model Based Statistical Learning/Enron.Rdata', probable reason 'No such file or directory'”


ERROR: ignored

## Part 2: Project objectives
### Reconstruct the email network from the raw data

To construct the email network we need to construct the edges and the nodes. 

An edge list is a data frame that contains a minimum of two columns, one column of nodes that are the source of a connection and another column of nodes that are the target of the connection. An edge list can also contain additional columns that describe attributes of the edges such as a magnitude aspect for an edge. If the edges have a magnitude attribute the graph is considered weighted.This is the case here, the weight corresponds to the number of email from the sender to the recipient  from 1999 to 2002.

The nodes in the data are identified by unique IDs. If the distinction between source and target is meaningful, the network is directed. If the distinction is not meaningful, the network is undirected. 

In the Enron dataset the distinction between source (sender) and target (rvalue) is clearly meaningful, and so the network is directed. 

To facilitate the network visualization I name the sender column as “from”, the target column as “to” and I use integers beginning with one as node IDs.  

##### Step 1: Construct the nodes

We test if we need the other email address. 

In [ ]:
dim(message$sender[message$sender %in% employeelist$Email2])
dim(message$sender[message$sender %in% employeelist$Email3])
dim(message$sender[message$sender %in% employeelist$EMail4]) #first we search among the senders

In [ ]:
dim(message$rvalue[message$rvalue %in% employeelist$Email2])
dim(message$rvalue[message$rvalue %in% employeelist$Email3])
dim(message$rvalue[message$rvalue %in% employeelist$EMail4]) #then among the recipients

This is never the case according to the previous code, so we will focus on the variable "Email id" from employee list to construct the network. 

In [ ]:
nodes <- employeelist %>% distinct(Email_id) %>%
  rename(label="Email_id")#149 persons
nodes <- nodes %>% rowid_to_column("id")
nodes <-  nodes %>% rename(label = "label")
nodes[1:5, ]

##### Step 2: Construct the edges

In [ ]:
edges_test <- merge(message, recipientinfo, by.x="mid", by.y="mid")

In [ ]:
## we delete the case when the sender or the receiver is not from the company
sub <- edges_test[(edges_test$rvalue %in% nodes$label), ]
sub <- sub[(sub$sender %in% nodes$label), ]
dim(sub)

As far as we focus on the company employees as nodes, we are not interested in the emails between employees and individuals not from the company. After removing them the number of edges is 69 407, we lost 1 995 035 mails with this process. 

In [ ]:
edges<- sub[,c(2,10)] #we keep only the columns sender (2) and rvalue (10)

In [ ]:
#Add a variable weight corresponding to the number of emails from sender to rvalue
per_route <- edges %>%  
  group_by(sender, rvalue) %>%
  summarise(weight = n()) %>% 
  ungroup()

In [ ]:
#Rename all the sender by their id in a new column
edges <- per_route %>% 
  left_join(nodes, by = c("sender" = "label")) %>% 
  rename(from = id)

In [ ]:
#Rename all the rvalue by their id in a new column
edges <- edges %>% 
  left_join(nodes, by = c("rvalue" = "label")) %>% 
  rename(to = id)

In [ ]:
#Keep only from to with the id and the weight
edges <- select(edges, from, to, weight)
edges[1:5,]

#### Step 3: Construct the network with the package network. 

In [ ]:
network_email <- network(edges, vertex.attr = nodes, matrix.type = "edgelist", ignore.eval = FALSE)

In [ ]:
network_email

So at the end we obtain we network of 149 nodes and 2582 weighted and directed egdes. 

#### Visualization of the network. 

With the package network

In [ ]:
plot(network_email) #, mode = "circle")

In this case, the only change that I make to the default plot() function for the network package is to increase the size of nodes with the vertex.cex argument to make the nodes more visible. Even with this very simple graph, we can already learn something about the data. We directly notice that 3 employees haven't any interaction by emails with the others. Outside of these 3 outliers, we seems to observe 3 groups with this graph. 

With the package igraph. 

To create an igraph object from an edge-list data frame we can use the graph_from_data_frame() function. 

In [ ]:
email_igraph <- graph_from_data_frame(d = edges, vertices = nodes$id, directed = TRUE)
plot(email_igraph, edge.arrow.size = 0.2)

In [ ]:
plot(email_igraph, layout = layout_with_graphopt, edge.arrow.size = 0.2)

With this packages we can not identify groups, however we still have the 3 "outliers" with the number 92 which seems to send an email to himself. 

With the tidy package. 

We first create a network object using tidygraph, which is called a tbl_graph. A tbl_graph consists of two tibbles: an edges tibble and a nodes tibble. Conveniently, the tbl_graph object class is a wrapper around an igraph object, meaning that at its basis a tbl_graph object is essentially an igraph object.

In [ ]:
email_tidy <- tbl_graph(nodes = nodes, edges = edges, directed = TRUE)
email_igraphroutes_igraph_tidy <- as_tbl_graph(email_igraph)

In [ ]:
email_tidy %>% 
  activate(edges) %>% 
  arrange(desc(weight))

In [ ]:
plot(email_tidy)

With the package ggraph

In [ ]:
ggraph(email_tidy, layout = "graphopt") + 
  geom_node_point() +
  geom_edge_link(aes(width = weight), alpha = 0.8) + 
  scale_edge_width(range = c(0.2, 2)) +
  geom_node_text(aes(label = label), repel = TRUE) +
  labs(edge_width = "Letters") +
  theme_graph()

For the previous graphs the weight was not considered on the visualization. Here the intensity of the writing corresponds to the weight. Unfortunaly the graph is unreadable in this way because we can not see where the nodes belongs. 

In [ ]:
nodes[nodes$id==92,]

In [ ]:
sub[sub$sender=='mary.fischer@enron.com', ]

The outliers are making sense: three employees do not have any interactions with the others and Mary Fisher sent an email to herself!

### Use the SBM method (using the mixer package) to cluster the network.


Estimate the parameters, the hidden class variables, as well as the number of classes q of a MIXture
of Erdös Rényi random graphs. The estimation is performed for binary graphs (edges are assumed
to be drawn from Bernoulli distributions).


In [ ]:
install.packages("https://cran.r-project.org/src/contrib/Archive/mixer/mixer_1.9.tar.gz",repos=NULL)

In [ ]:
library(mixer)

https://cran.microsoft.com/snapshot/2016-02-10/web/packages/mixer/mixer.pdf 

In [ ]:
X=as.matrix.network.adjacency(network_email)

In [ ]:
out=mixer(X,qmin=3,qmax=3)
plot(out)